# Seleção de Modelos: Avaliação e Validação

In [186]:
import os
import joblib
import pandas as pd
from dotenv import load_dotenv
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.metrics import classification_report

In [187]:
load_dotenv()
random_state = int(os.environ['SKLEARN_RANDOM_STATE'])

print(os.environ['NOTEBOOKS_PROCESSED_DATA_PATH'])
print(os.environ['SKLEARN_RANDOM_STATE'])

../data/processed/
42


## Carregando os dados

In [6]:
df = pd.read_parquet(
    os.environ['NOTEBOOKS_PROCESSED_DATA_PATH'] + \
        'creditcard_train.parquet'
)

df.shape

(227845, 31)

In [8]:
X = df.drop('Fraude', axis=1).values
y = df['Fraude'].values

X.shape, y.shape

((227845, 30), (227845,))

## Balanceamento de classes

## Normalização

## Selecionando os modelos com validação cruzada

In [129]:
models = {
    'qda': QuadraticDiscriminantAnalysis(),
    'gnb': GaussianNB(),
    'xgb': XGBClassifier(random_state=random_state),
    'rf': RandomForestClassifier(random_state=random_state)
}

In [130]:
kfold = StratifiedKFold(n_splits=5)

In [55]:
%%time

reports = []

for model_name, model in models.items():
    report = []
    for i, (train_index, val_index) in enumerate(kfold.split(X, y)):
        model.fit(X[train_index], y[train_index])
        y_pred = model.predict(X[val_index])

        partial_report = classification_report(
            y[val_index],
            y_pred,
            output_dict=True
        )

        partial_report = pd.DataFrame(partial_report)
        report.append(partial_report)

    report = sum(report) / len(report)
    reports.append(report)
    reports = pd.concat(reports)

Wall time: 14min 47s


In [128]:
index_tuples = [
    [
        (model_name, 'precision'),
        (model_name, 'recall'),
        (model_name, 'f1-score'),
        (model_name, 'support')
    ] for model_name in list(models.keys())
]

index_tuples = [index for level in index_tuples \
                      for index in level]

index = pd.MultiIndex.from_tuples(
    index_tuples,
    names=['model', 'metric']
)

reports = reports.set_index(index)
reports = reports.sort_values(by=['accuracy'], ascending=False)
reports

0          1  accuracy     macro avg  weighted avg
model metric                                                                  
rf    precision      0.999925   0.778673  0.999535      0.889299      0.999535
      recall         0.999767   0.854535  0.999535      0.927151      0.999511
      f1-score       0.999609   0.947904  0.999535      0.973756      0.999518
      support    45488.600000  80.400000  0.999535  45569.000000  45569.000000
xgb   precision      0.999921   0.778704  0.999530      0.889312      0.999530
      recall         0.999765   0.853402  0.999530      0.926584      0.999506
      f1-score       0.999609   0.945437  0.999530      0.972523      0.999513
      support    45488.600000  80.400000  0.999530  45569.000000  45569.000000
gnb   precision      0.999363   0.146781  0.992719      0.573072      0.997859
      recall         0.996342   0.238640  0.992719      0.617491      0.995005
      f1-score       0.993339   0.641790  0.992719      0.817565      0.992719
      support    45488.600000  80.400000  0.992719  45569.000000  45569.000000
qda   precision      0.999770   0.059960  0.975602      0.529865      0.998112
      recall         0.987630   0.112194  0.975602      0.549912      0.986086
      f1-score       0.975783   0.873025  0.975602      0.924404      0.975602
      support    45488.600000  80.400000  0.975602  45569.000000  45569.000000

## Salvando o melhor modelo

In [190]:
joblib.dump(
    models['xgb'],
    os.environ['NOTEBOOKS_MODELS_PATH'] + \
        'xgb_fraud_detection_train.sav'
)

['../models/xgb_fraud_detection_train.sav']